## Exploring RAG

In [1]:
import os
import openai


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"




In [3]:

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

MODEL = "gpt-3.5-turbo"
#MODEL = "llama3"
#MODEL = "mistral"

## Chat API : OpenAI
Let's start with a direct API calls to OpenAI.

In [ ]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

get_completion("What is 1+1?")

In [5]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat = ChatOpenAI(
    openai_api_key = os.environ['OPENAI_API_KEY'],
    model='gpt-3.5-turbo'
)

/home/tema/10X/week11/we11env/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Bot: Goodbye!")
        break
    
    # Get response from the model
    response = chat.send_message(user_input)
    print("Bot:", response)

## using Langchain

In [6]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings


if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
elif MODEL in ["mistral", "llama3"]:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)
else:
    raise ValueError(f"Unsupported model: {MODEL}")  # Handle unsupported models

ChatOpenAI(temperature=0.0, model=llm_model)

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7a0f5d516c60>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7a0f5d5300e0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [8]:
model.invoke("what is the square root of 9")

AIMessage(content='The square root of 9 is 3.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 15, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-244310d8-b2f3-4c22-8203-e40982bf483d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 10, 'total_tokens': 25})

In [9]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

In [11]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [12]:
response = model.invoke(prompt)
print(response)

content="Ah, I am quite upset that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help at this moment, my friend." response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 89, 'total_tokens': 143}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-f6e3267b-9415-407b-9169-50dfd6d87dd8-0' usage_metadata={'input_tokens': 89, 'output_tokens': 54, 'total_tokens': 143}


### Langchain Models, prompts and parsers

In [23]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=MODEL)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x71182cbf9c70>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x71182cbc2540>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [16]:
#define prompt template

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [24]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [25]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [26]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [29]:
customer_style = """American English \
in a calm and respectful tone
"""

In [27]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [30]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [31]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [32]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"
